### Initialize tracking emulating how SIENTIA™ website environment works

In [17]:
# import sientia_tracker.simple_tracker as simple_tracker
from sientia_tracker import simple_tracker


# Initialize tracking
tracking_uri = "http://localhost:57398/"
username = "aignosi"
password = "aignosi"
project_name = "teste_logs"

tracker = simple_tracker.SimpleTracker(tracking_uri,username,password)
tracker.set_project(project_name)



Experiment teste_logs already exists


### Load the dataset and create a model

In [18]:
# Importações necessárias do scikit-learn, pandas e outros pacotes
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


prediction_params = {
    "max_iter": 1000
}


# Classe para pré-processamento de dados
class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, params=None):
        # Inicializa a classe com parâmetros opcionais
        self.params = params
        self.model = None
        self.metrics = {}

    # Método fit, geralmente usado para ajustar parâmetros, mas aqui só aplica a transformação
    def fit(self, x, y=None):
        self.transform(x)  # Aplica a transformação (remoção de valores nulos)
        return self  # Retorna a própria instância

    # Método de transformação que remove valores ausentes (NaN) dos dados
    def transform(self, x):
        x = x.dropna()  # Remove valores nulos
        return x  # Retorna os dados transformados

    # Método predict que apenas aplica a transformação
    def predict(self, x):
        return self.transform(x)  # Aplica a transformação aos dados de entrada


# Classe para o modelo de predição
class PredictionModel(BaseEstimator, TransformerMixin):
    def __init__(self, params=None):
        # Inicializa com parâmetros opcionais
        self.params = params
        self.model = None  # Inicialmente, o modelo é None
        self.metrics = {}

    # Método fit, usado para treinar o modelo
    def fit(self, x, y=None):
        # Treina um modelo de Regressão Logística
        model = LogisticRegression(max_iter=self.params['max_iter'])  # Define o modelo com no máximo 1000 iterações
        model.fit(X_train, y_train)  # Ajusta o modelo aos dados de treino
        self.model = model  # Armazena o modelo treinado
        return self.model  # Retorna o modelo treinado

    # Método predict, usado para fazer previsões
    def predict(self, x):
        # Faz predições com base no modelo treinado
        y_pred = self.model.predict(x)  # Aplica o modelo para prever os rótulos de x
        return y_pred  # Retorna as predições


# Instancia as classes de pré-processamento e modelo
transform_model = DataPreprocessor()
pred_model = PredictionModel(prediction_params)

# Carrega o conjunto de dados 'Iris'
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)  # Converte para DataFrame do pandas
df['target'] = data.target  # Adiciona a coluna 'target' (rótulos)

# Aplica o pré-processamento aos dados
transform_model.fit(df)  # Aplica o método fit (mesmo que não faça nada além de transformar)
df = transform_model.transform(df)  # Aplica a transformação (remoção de valores nulos)

# Separa os dados em variáveis independentes (X) e dependentes (y)
X = df.drop(columns=['target'])  # Remove a coluna 'target'
y = df['target']  # Define os rótulos como a variável 'target'

# Divide os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # 80% treino, 20% teste

# Treina o modelo de predição com os dados de treino
pred_model.fit(X_train, y_train)



acc_score = accuracy_score(pred_model.predict(X_test), y_test)
acc_score

0.9

### Initialize run. The only needed parameters are the experiment name and inputs

In [19]:
# Start a run
dataset_name= "Iris"
inputs= "sepal_length, sepal_width, petal_length, petal_width"
run = tracker.save_experiment(dataset_name=dataset_name, inputs=inputs)
run_id = run.info.run_id


Saving experiment teste_logs


### Log models and metrics

In [20]:
pred_model_atributes = vars(pred_model)
transform_model_atributes = vars(transform_model)


#log parameters and metrics
tracker.log_params(pred_model_atributes['params'])
tracker.log_metrics({"accuracy": acc_score})

# Log models
tracker.log_model(transform_model, 'data_model')
tracker.log_model(pred_model, "prediction_model")

### Retrieve information of run

In [ ]:
# Retrieve the run using the run ID
retrieved_run = tracker.client.get_run(run_id)

# Access and print metrics and params 
metrics = retrieved_run.data.metrics
params = retrieved_run.data.params
print("Metrics:", metrics)
for key, value in params.items():
    print( key,':' ,value)

